In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


from sklearn.model_selection import cross_val_score, KFold

from sklearn.metrics import f1_score

import pickle

In [11]:
train_test_df = pd.read_csv('alfabattle2_abattle_train_target.csv')
submit_df = pd.read_csv('alfabattle2_prediction_session_timestamp.csv')

In [3]:
SEED = 1

## EDA

In [4]:
parquet = pd.read_parquet('/data/edmitrie/alfa_boosters/parquets/part-00000.parquet')

In [5]:
parquet.head(n=5)

,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
0,2020-01-15 08:46:32.579,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,pv,None,None,None,None,Asia/Omsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None
1,2020-04-29 19:20:09.705,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,pv,None,None,None,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None
2,2020-02-21 03:21:24.058,anketa,489818f57087b59790435b3acf18018a,8bae6c4249db74dc0937e1ff00d61079,pv,None,None,None,None,Asia/Yakutsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None
3,2020-08-06 06:54:30.918,anketa,489818f57087b59790435b3acf18018a,3a95ba68795b745ca09ebee138b418ef,pv,None,None,None,None,Asia/Yakutsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None
4,2020-02-28 18:51:56.452,anketa,6dde48ffeb6d354628303c9d9b6721be,1db024ef087161f30a3c981cb32bb565,pv,None,None,None,None,Asia/Omsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None


In [8]:
parquet[parquet.session_id == '9a1366886da59ec85f0ef5e06f1e97f6'].tail(n=50)

,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
55750,2020-01-15 08:46:32.587,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,Non-interaction,show,51c65b92f640d5892d452468d4ed5fef,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
55751,2020-01-15 08:46:32.806,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,Non-interaction,show,51c65b92f640d5892d452468d4ed5fef,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
57072,2020-01-15 08:46:52.559,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,click,f4f1845568f67cab7b66f93102d636c9,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
57073,2020-01-15 08:46:52.570,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,click,f4f1845568f67cab7b66f93102d636c9,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
58080,2020-01-15 08:46:39.249,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,fieldFilling,6572b2d86a67755a968b9272f14f8bf7,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
58081,2020-01-15 08:46:39.253,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,fieldFilling,6572b2d86a67755a968b9272f14f8bf7,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
58682,2020-01-15 08:46:52.378,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,fieldFilling,8a4965c45a769d365507ff0adb5202e0,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
58683,2020-01-15 08:46:52.387,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,interactions,fieldFilling,8a4965c45a769d365507ff0adb5202e0,None,Asia/Omsk,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None
59593,2020-01-15 08:56:36.257,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,Ecommerce,View,667c8b9e47bcb5e44ae25d66ba567651,None,Asia/Omsk,True,anketa.alfabank.ru,e470f4a7f3a58196401e9e218af4ffbe,None,None
60017,2020-01-15 08:56:36.229,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,se,Pil,Anketa::Step2::Send,7023e063742223c292f37332febf1fb0,None,Asia/Omsk,True,anketa.alfabank.ru,e470f4a7f3a58196401e9e218af4ffbe,None,None


In [6]:
parquet['timestamp'] = pd.to_datetime(parquet['timestamp'])

In [7]:
parquet.describe()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
count,12468595,12468595,12468595,12468595,12468595,6766054,6682190,1751796,9665011,12468595,12243467,437725,437725,12030870,7000769
unique,12205780,23,8211,1004032,5,334,10397,6369,535,169,1,10,14207,3,14
top,2020-01-09 06:51:16,mobile,ab3851b3743e3c68a0e734262afa3a68,2e5dd1eef55786a3a161b45375bfae25,se,Application Lifecycle,Foreground,7a4e59a5d9e3bf6484ec3ca5e9f7e277,SignInActivity,Europe/Moscow,True,online.alfabank.ru,fbb19948fb982a9efafde85cf39ed6ed,mobile,LTE
freq,20,12030870,28570,4683,6766070,2118633,1136388,760988,1225890,7056856,12243467,201946,22793,6998385,5781267
first,2020-01-01 00:00:40.377000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-09-30 22:35:35.518000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
parquet['event_type'].unique()

array(['pv', 'se', 'err', 'sv', 'ue'], dtype=object)

## Просмотр сессий без таргета

In [14]:
res = pd.merge(parquet, train_test_df, on='session_id', how='left')

In [21]:
null_target = res[res['multi_class_target'].isnull()]
not_null_target = res[res['multi_class_target'].notnull()]

In [19]:
null_target[null_target['session_id'] == 'bdf7fc21e4e88d1b4f0838c80b21a1e3'].groupby('session_id')

,timestamp_x,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech,client_pin,timestamp_y,multi_class_target
1,2020-04-29 19:20:09.705,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,pv,None,None,None,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None,NaN,NaN,NaN
460,2020-04-29 19:22:05.451,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,pv,None,None,None,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
5756,2020-04-29 19:20:09.925,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,pv,None,None,None,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,d0a697abf7c744a109a1c0ab4d052ef7,None,None,NaN,NaN,NaN
10261,2020-04-29 19:22:45.733,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,Credit card,Anketa::Step2::Send,8011fa5d87e14befc1a943028ecf6b8a,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
10262,2020-04-29 19:22:54.790,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,Credit card,Anketa::Step2::Send,8011fa5d87e14befc1a943028ecf6b8a,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
10576,2020-04-29 19:22:45.708,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,Credit card,Anketa::Персональные данные::Click,2715a83ce3fbdcf983c27da75ab341f3,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
10577,2020-04-29 19:22:54.890,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,Credit card,Anketa::Персональные данные::Click,2715a83ce3fbdcf983c27da75ab341f3,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
11025,2020-04-29 19:22:05.597,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,Ecommerce,View,fdc0ddf26f580767e08c89b85cf00f3a,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
11772,2020-04-29 19:22:05.761,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,interactions,click,866bc42c5bed56cebc08a4df83336c11,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN
11774,2020-04-29 19:22:05.821,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,se,interactions,click,866bc42c5bed56cebc08a4df83336c11,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,3c932fa03e6ee7f92b6a5a854d885c13,None,None,NaN,NaN,NaN


In [33]:
not_null_target[not_null_target['multi_class_target'] == 'invest'].groupby('session_id').count()

,timestamp_x,application_id,client,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech,client_pin,timestamp_y,multi_class_target
session_id,,,,,,,,,,,,,,,,,
00103c645d5cec8128e3553ba1fa6832,20,20,20,20,8,8,3,20,20,20,0,0,20,0,20,20,20
00177540b679b4c94d6e27657b3bede9,17,17,17,17,12,12,6,15,17,17,0,0,17,17,17,17,17
0020f1d852b445c1dc7dea06af69c30f,29,29,29,29,20,20,15,27,29,29,0,0,29,0,29,29,29
00271909d81f67a8202f454111ffb21b,98,98,98,98,57,57,42,82,98,98,12,12,86,0,98,98,98
0027a29b233b5725eb4f82a583c5e088,103,103,103,103,44,44,20,99,103,103,0,0,103,103,103,103,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc8f6c0a4220582a323a6df64715302,71,71,71,71,44,44,35,65,71,71,6,6,65,0,71,71,71
ffd2cf16f2fcf06d13423fcf97d6da93,26,26,26,26,14,14,11,26,26,26,0,0,26,0,26,26,26
ffd831a9caa20865d03fb3372e60de91,31,31,31,31,19,19,11,27,31,31,0,0,31,31,31,31,31


In [34]:
not_null_target[not_null_target.session_id == '00103c645d5cec8128e3553ba1fa6832'].sort_values(by='timestamp_x').head(n=50)

,timestamp_x,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech,client_pin,timestamp_y,multi_class_target
11881194,2020-02-12 14:51:09.622,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,sv,None,None,None,SignInActivity,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
10797645,2020-02-12 14:51:15.771,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,sv,None,None,None,MainListFragment,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
5778595,2020-02-12 14:51:39.508,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,se,Main Screen,My Finances click,None,Main list,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
11323659,2020-02-12 14:51:39.684,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,sv,None,None,None,PfmMainActivity,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
10797647,2020-02-12 14:51:47.351,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,sv,None,None,None,MainListFragment,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
5770903,2020-02-12 14:51:52.765,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,se,Main Screen,Investments click,None,Main list,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
10311268,2020-02-12 14:51:53.021,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,sv,None,None,None,InvestmentsMainActivity,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
5345527,2020-02-12 14:51:54.206,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,se,Investments,Load main PIFs,05dfdf6b29e3e731a712f060c9cf8896,Investments Main Screen,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
5330429,2020-02-12 14:51:54.236,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,se,Investments,Load individual accounts catalogue,05dfdf6b29e3e731a712f060c9cf8896,Investments Main Screen,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest
5337925,2020-02-12 14:51:54.707,mobile,f8f3e358c6dc12f8a1329865db3af929,00103c645d5cec8128e3553ba1fa6832,se,Investments,Load individual accounts catalogue,7a4e59a5d9e3bf6484ec3ca5e9f7e277,Investments Main Screen,Asia/Yekaterinburg,True,None,None,wifi,None,f8f3e358c6dc12f8a1329865db3af929,2020-02-12 14:51:09,invest


### количество таймзон

In [286]:
time_zones = parquet.timezone.unique()

In [287]:
time_zones.shape

(169,)

In [288]:
time_zones[:100]

array(['Asia/Yekaterinburg', 'Europe/Moscow', 'Asia/Krasnoyarsk',
       'Asia/Shanghai', 'Europe/Samara', 'Europe/Minsk', 'Asia/Omsk',
       'Asia/Dubai', 'Europe/Saratov', 'Asia/Irkutsk', 'Asia/Novosibirsk',
       'Europe/Volgograd', 'Europe/Berlin', 'Asia/Kolkata',
       'Asia/Ho_Chi_Minh', 'Europe/Kiev', 'Asia/Vladivostok',
       'Europe/Helsinki', 'Europe/Rome', 'Europe/Athens', 'Asia/Sakhalin',
       'Asia/Bishkek', 'Asia/Makassar', 'Asia/Bangkok',
       'Asia/Novokuznetsk', 'Europe/Kaliningrad', 'Asia/Almaty',
       'Asia/Barnaul', 'Asia/Magadan', 'Asia/Calcutta', 'Asia/Kamchatka',
       'Asia/Tashkent', 'Asia/Colombo', 'Asia/Karachi',
       'Antarctica/Macquarie', 'Asia/Saigon', 'Asia/Aqtau',
       'Europe/Budapest', 'Asia/Baku', 'Europe/Amsterdam',
       'Europe/London', 'Asia/Oral', 'Asia/Jerusalem', 'Pacific/Midway',
       'Europe/Chisinau', 'Europe/Madrid', 'GMT+05:00', 'Asia/Yerevan',
       'Asia/Tbilisi', 'Asia/Kuwait', 'Asia/Yakutsk', 'Europe/Brussels',
    

### просмотр event_category

In [115]:
train_test_df['multi_class_target'].unique()

array(['main_screen', 'invest', 'statement', 'phone_money_transfer',
       'own_transfer', 'credit_info', 'chat', 'card2card_transfer',
       'mobile_recharge', 'card_recharge'], dtype=object)

In [167]:
parquet[parquet['event_category'].notnull()]['event_category'].sort_values().unique()[:100]

array(['100-days-draft', '::credit_card', '@@router/location-change',
       'ATMs And Offices', 'Account Closing', 'AccountDetails',
       'Accounts Cards', 'Action menu', 'Activation', 'Add passcode',
       'Address Suggestions', 'Advice', 'Affluent Feedback', 'AlfaCheck',
       'AlfaTravel', 'All Payments', 'AllProviders', 'AppStoreReview',
       'Application Lifecycle', 'Auth', 'AutoPayments',
       'Balance Transfer', 'BalanceAppWidget', 'Bank Alerts',
       'Bank communication feedback', 'Bank_Offer',
       'Biometrics Collection', 'Bond Page', 'BonusTransfers', 'Bonuses',
       'Bookkeeping', 'Bottom Navigation', 'Buttons', 'C2C',
       'Card Reissuing', 'Card To Card Transfer', 'Card details',
       'CardRequisites', 'Cards', 'Cards Closing', 'CardsActivation',
       'Carousel', 'CataloguePayment', 'Cc-short', 'Chat',
       'ClientPhoneTransfer', 'Communal Payments', 'Communal payments',
       'Complete Page', 'Confirmation',
       'Consent For Processing Personal

In [214]:
res = parquet[parquet['event_category'] == 'Chat']
res = res.merge(train_test_df[['multi_class_target', 'session_id']], on='session_id')[['multi_class_target', 'session_id']]
res.drop_duplicates(inplace=True)

In [215]:
res['multi_class_target'].value_counts()

chat                    16148
credit_info              1752
own_transfer              461
phone_money_transfer      398
card2card_transfer        372
card_recharge             251
invest                    240
mobile_recharge           168
Name: multi_class_target, dtype: int64

In [281]:
parquet['event_type'].unique()

array(['sv', 'se', 'pv', 'ue', 'err'], dtype=object)

### просмотр event_name

In [173]:
names = parquet['event_name'].unique()

In [182]:
names[:200]

array([None, 'Home screen tab selected', 'Click Rates Section',
       'Click Notification in main menu', 'INSURANCE',
       'Account item click', 'Click > Statements screen Tab',
       'load statement bonus', 'Generated CC', 'Recharge click',
       'Accounts Cards click', 'Expand Accounts', 'Click Card Details',
       'Collapse Accounts', 'Force refresh', 'Click > Main list Tab',
       'Click CC', 'event', 'Click Account Details', 'Open Spotlight',
       'Click out Highlighted View', 'beaeebce09f041203e85e007b277db60',
       'Load card info', 'Start drag and drop', 'Preselect Accounts',
       'Suggest Click', 'Mobile Recharge click', 'Mobile Recharge',
       'Simple screen show template', 'Simple screen auto payment show',
       'Simple screen show todo', 'Generated PIL', 'Advice action click',
       'Open detail', 'click', 'open Support Chat screen from push',
       '4b08db0f4c83768db095e4f5d7d1b6b5',
       'Request History Result > Success', 'Hints Result > Not Found',


In [219]:
res = parquet[parquet['event_name'] == 'Mobile Recharge click']

In [220]:
res = res.merge(train_test_df[['multi_class_target', 'session_id']], on='session_id')[['multi_class_target', 'session_id']]
res.drop_duplicates(inplace=True)

In [221]:
res['multi_class_target'].value_counts()

mobile_recharge         27445
phone_money_transfer     1227
own_transfer              726
card2card_transfer        493
credit_info               419
chat                      245
card_recharge             110
invest                     74
statement                   1
Name: multi_class_target, dtype: int64

### просмотр event_labdf_all

In [229]:
names = parquet['event_labdf_all'].unique()
names.shape

(6370,)

In [224]:
names[:200]

array([None, '3fa2d0d9f3cc1cc05b70512aa57ec208',
       '39ad6923f37a0f9e718290e815edef1c',
       '7a4e59a5d9e3bf6484ec3ca5e9f7e277',
       'a7e467750a990887bfeb95f9384e146b',
       '428118f7c971a68d01d0f32b61fcc5d3',
       'bf5f79050f13d29d738cb2413d5d1296',
       '20c672ef91378f866fbfdc78425e1b7d',
       '24f96f775c5236e0f7594ef40cc86666',
       '3516c8894167c7fa9c1d6256e4074e23',
       '93ef4038c53c9356b3156f8e60c60f53',
       'd4b5caf39100c8a37d3b5ed5af06cbc1',
       '4cf78e083331d1c9bddae59007639e68',
       '77556759755856506c6878c924f918b9',
       '05dfdf6b29e3e731a712f060c9cf8896',
       'c43525954bbafcd8d4c122bdf67b2a5f',
       '5054e107585cbce53eb38fec52735187',
       'de3849a224f9d88f6495d467c163c2fd',
       '5b14bf081cdfdbd3b37a3acb75bc553c',
       'ebfc2d87006456f781a1c5c39d158a2d',
       'acb58a48d6bda9f2a2a1f9475652660c',
       '2744c914793579c40c6815aff37c4a7e',
       '589a5a2cce0bb27f715e32da624af275',
       'ba4e01babeb853bb53f84a948c3973e5',
     

In [240]:
res = parquet[parquet['event_labdf_all'] == 'fe9a2ea9d6e5e445507ad32e8c5504ae']
res = res.merge(train_test_df[['multi_class_target', 'session_id']], on='session_id')[['multi_class_target', 'session_id']]
res.drop_duplicates(inplace=True)

res['multi_class_target'].value_counts()

phone_money_transfer    6393
own_transfer             553
card2card_transfer       537
statement                236
mobile_recharge          133
chat                     100
credit_info               91
card_recharge             58
invest                    16
Name: multi_class_target, dtype: int64

In [239]:
parquet['event_labdf_all'].value_counts().head(n=10)

7a4e59a5d9e3bf6484ec3ca5e9f7e277    760988
77556759755856506c6878c924f918b9     74353
a7e467750a990887bfeb95f9384e146b     61567
4cf78e083331d1c9bddae59007639e68     60186
4930579a044a6143726f5a2dfca83203     52778
05dfdf6b29e3e731a712f060c9cf8896     36877
93ef4038c53c9356b3156f8e60c60f53     33203
007ab75be96e71f4c830649fb1fb3a8a     30807
fe9a2ea9d6e5e445507ad32e8c5504ae     30237
24f96f775c5236e0f7594ef40cc86666     21863
Name: event_label, dtype: int64

In [234]:
parquet['event_labdf_all'].value_counts().head(n=100)

7a4e59a5d9e3bf6484ec3ca5e9f7e277    760988
77556759755856506c6878c924f918b9     74353
a7e467750a990887bfeb95f9384e146b     61567
4cf78e083331d1c9bddae59007639e68     60186
4930579a044a6143726f5a2dfca83203     52778
                                     ...  
c5002fb225ee3fb7e330c7f7b211402f      1159
79dc8e98a4f0075ade13c34f1d80fb94      1157
ee8d19560b44dfbdf51f0cc39fde248e      1135
5cac9555bbbd319b794afcdbd714c023      1133
5819b28ad266c6523d292b57f8eb5f5f      1128
Name: event_label, Length: 100, dtype: int64

### работа с таймзонами

In [274]:
res = parquet.groupby('session_id')['timezone'].unique()

In [278]:
df_many_timezones = res[res.apply(lambda x: len(x) > 2)]

In [279]:
df_many_timezones

session_id
06fba24be26c7393b68bac7072a159c3          [Asia/Krasnoyarsk, Asia/Omsk, Asia/Jakarta]
1613dd17c27d0a3780c247ad36777c7c           [Europe/Samara, Asia/Dubai, Europe/Moscow]
16ccff2954251e039577bd787929d8a9          [Asia/Baghdad, Europe/Moscow, Europe/Minsk]
1bf1612f1b8cabcfd3c4dc2748b9ce6e       [Europe/Samara, Europe/Saratov, Europe/Moscow]
1c7f3cb77ca32c03d84a3df9fe6713b5     [Europe/Minsk, Asia/Yekaterinburg, Asia/Karachi]
1eb6d5544cfc42d637ba3658015d6dec      [Asia/Krasnoyarsk, Europe/Moscow, Europe/Minsk]
2b5d55ee158bc64aaa76ddc8923e3516          [Africa/Cairo, Europe/Moscow, Europe/Minsk]
2bf0f817cea20829bfc093b9075f98ad      [Europe/Moscow, Europe/Volgograd, Europe/Minsk]
30f51f88cd549895e2d9a1f1b13cc75b    [Asia/Vladivostok, Asia/Yakutsk, Australia/Bri...
3df80f6136f84e062d3e8b5e9c6d28e0    [Asia/Yekaterinburg, Europe/Moscow, Europe/Minsk]
43603c5e2d965872fb2433675235581d          [Asia/Baghdad, Europe/Moscow, Europe/Minsk]
45d8e8855841109fdf158e290555cb10          [

In [276]:
df_many_timezones.shape

(24688,)

In [78]:
train_test_df.join(df_many_timezones, on='session_id', how='inner')['multi_class_target'].value_counts()

credit_info             2365
statement               2055
chat                    1632
own_transfer             912
phone_money_transfer     429
invest                   422
card2card_transfer       372
mobile_recharge          366
card_recharge            366
main_screen               33
Name: multi_class_target, dtype: int64

## Feature engeneering

In [7]:
parquet['timestamp'] = pd.to_datetime(parquet['timestamp'])

In [8]:
atrs = tuple([col for col in parquet.columns if col not in ['session_id', 'client']])

In [10]:
def gen_features_2(res):
    atrs = [col for col in res.columns if col not in ['session_id', 'timestamp','client_pin']]
    
    res['timestamp_first'] = pd.to_datetime(res['timestamp_first'])
    
    res.sort_values(by='timestamp_first', inplace=True)
    
    res[atrs] = res.groupby('client_pin')[atrs].shift(1)
    
    atrs_for_mean = ['session_diff_time', 'session_id_size']
    
    res['sum_session_size'] = res.groupby('client_pin')['session_id_size'].cumcount()
    
    for atr in atrs_for_mean:
        res[f'{atr}_sum'] = res.groupby('client_pin')[atr].cumsum()
        res[f'{atr}_mean'] = res[f'{atr}_sum'] / res['sum_session_size'] 
        res[f'{atr}_explotation'] = res[atr] / res[f'{atr}_mean'] 
           
    return res

### Обучений подклассификатора для сессий

In [14]:
parquet = pd.read_parquet('/data/edmitrie/alfa_boosters/parquets/part-00000.parquet')

In [15]:
parquet.head(n=1)

,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
0,2020-01-15 08:46:32.579,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,pv,None,None,None,None,Asia/Omsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None


In [16]:
parquet.loc[parquet['device_is_webview'].isnull(), 'device_is_webview'] = False

In [4]:
import pickle

dict_cols_categ = pickle.load(open('dict_usefull_ctgrs','rb'))

In [10]:
dict_cols_categ

{'application_id': ['mobile',
  'retail_upsale_ui',
  'retail_upsale_cards',
  'site',
  'retail_realty_insurance',
  'retail_cards_activation',
  'anketa',
  'retail_cards_insurance_antifraud_sale',
  'retail_cards_pin_change',
  'retail-marketplace',
  'retail-taxes-ui',
  'retail_upsale_verification',
  'retail_debit_card_order'],
 'event_type': ['pv', 'sv', 'se', 'ue'],
 'event_category': ['nan',
  'Application Lifecycle',
  'Credit Info',
  'SingleStatement',
  'Bottom Navigation',
  'All Payments',
  'Operations',
  'ResultScreen',
  'Card To Card Transfer',
  'Main Screen',
  'Chat',
  'Recharge',
  'ClientPhoneTransfer',
  'Push',
  'Advice',
  'Investments',
  'Widget Dashboard',
  'Confirmation',
  'Payment Hub',
  'ExternalPhoneTransfer',
  'Bank_Offer',
  'Phone Bank Picker',
  'Accounts Cards',
  'Phone Bank Transfer Confirmation',
  'DeepLink',
  'Notification Center push',
  'Bonuses',
  'Investments Bonds',
  'AppStoreReview',
  'ReverseCashBack'],
 'event_name': ['nan'

In [18]:
# def gen_features(df, category_dict):
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     df.sort_values(by='timestamp', inplace=True)
    
#     buf = pd.get_dummies(df['event_type'])    
#     df = pd.concat([df, buf], axis=1)
#     event_cols = buf.columns.values.tolist()
    
#     atrs = [col for col in df.columns if col not in ['session_id', 'client'] and col not in event_cols]
        
#     res = df.groupby(['session_id','client']).agg({**{atr:['first', 'last'] for atr in atrs},\
#                                                    'session_id':'size',\
#                                                    **{atr:['mean'] for atr in event_cols},\
#                                                   }\
#                                                  ).reset_index()
        
#     res.columns = ['session_id', 'client'] + [col[0] + '_' + col[1] for col in res.columns[2:]]
    
#     print(np.sum(res['timestamp_first'] > res['timestamp_last']))
    
#     res.set_index(['session_id', 'client'], inplace=True)
    
#     print('Зкончил 1 агрегирование')
            
#     df.fillna('nan', inplace=True)
    
#     for col in category_dict:
#          df.loc[df[col].apply(lambda x: x not in category_dict[col]), col] = 'other'
            
#     cat_cols = [col for col in category_dict]
    
#     buf = pd.get_dummies(df[cat_cols + ['session_id', 'client']], columns=cat_cols)
#     buf.columns = [df_all.replace(' ', '_') for df_all in buf.columns]
    
#     cols_to_agg = [col for col in buf.columns if col not in ['session_id', 'client']]
    
#     res_2 = buf.groupby(['session_id','client']).agg({atr:['mean'] for atr in cols_to_agg}).reset_index()  
    
#     res_2.columns = ['session_id', 'client'] + [col[0] + '_' + col[1] for col in res_2.columns[2:]]
    
#     res_2.set_index(['session_id', 'client'], inplace=True)
        
#     print('Зкончил 2 агрегирование')
        
#     res = res.join(res_2)
    
#     print('Зкончил merge')
        
#     res['session_diff_time'] = (res['timestamp_last'] - res['timestamp_first'])/np.timedelta64(1, 's')
    
#     print(res.shape)
    
#     def f(df):
#         keys, values = df.sort_values('session_id').values.T
#         ukeys, index = np.unique(keys, True)
#         arrays = np.split(values, index[1:])
#         df2 = pd.DataFrame({'session_id':ukeys, 'timezone':[list(a) for a in arrays]})
        
#         return df2
        
#     buf = f(df[['session_id', 'timezone']].drop_duplicates())
    
#     buf['timezone_num'] = buf['timezone'].apply(len)
    
#     buf['timezone'] = buf['timezone'].apply(lambda x: [df_all.strip() for df_all in x])
#     buf['timezone'] = buf['timezone'].apply(lambda x: ", ".join(set(x)))
    
#     for cat in category_dict:
#         res.loc[res[f'{cat}_first'].apply(lambda x: x not in category_dict[cat]) & res[f'{cat}_first'].notnull(), f'{cat}_first'] = 'other'
#         res.loc[res[f'{cat}_last'].apply(lambda x: x not in category_dict[cat]) & res[f'{cat}_first'].notnull(), f'{cat}_last'] = 'other'

#     res = res.join(buf.set_index('session_id'))
    
#     res.reset_index(inplace=True)
#     res.rename(columns={'client':'client_pin'}, inplace=True)
            
#     return res

In [77]:
def gen_features(df, category_dict):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(by='timestamp')
       
    atrs = [col for col in df.columns if col not in ['session_id', 'client']]
        
    res = df.groupby(['session_id','client']).agg({**{atr:['first', 'last'] for atr in atrs},\
                                                   'session_id':'size',\
                                                  }\
                                                 ).reset_index()
        
    res.columns = ['session_id', 'client'] + [col[0] + '_' + col[1] for col in res.columns[2:]]
        
    res.set_index(['session_id', 'client'], inplace=True)
    
    for cat in category_dict:
        res.loc[res[f'{cat}_first'].apply(lambda x: x not in category_dict[cat]) & res[f'{cat}_first'].notnull(), f'{cat}_first'] = 'other'
        res.loc[res[f'{cat}_last'].apply(lambda x: x not in category_dict[cat]) & res[f'{cat}_first'].notnull(), f'{cat}_last'] = 'other'
    
    print('Зкончил 1 агрегирование')
            
    df.fillna('nan', inplace=True)
    
    for col in category_dict:
         df.loc[df[col].apply(lambda x: x not in category_dict[col]), col] = 'other'
            
    
    cat_cols = [col for col in df.columns if col not in ['client', 'session_id', 'timestamp', 'timezone', 'device_is_webview']]
    
    buf = pd.get_dummies(df[cat_cols + ['session_id', 'client','device_is_webview']], columns=cat_cols)
    buf.columns = [col.replace(' ', '_') for col in buf.columns]
    
    cols_to_agg = [col for col in buf.columns if col not in ['session_id', 'client']]

    res_2 = buf.groupby(['session_id','client']).agg({atr :['mean'] for atr in cols_to_agg}).reset_index()
    
    res_2.columns = ['session_id', 'client'] + [col[0] + '_' + col[1] for col in res_2.columns[2:]]
    
    res_2.set_index(['session_id', 'client'], inplace=True)
        
    print('Зкончил 2 агрегирование')
        
    res = res.join(res_2)
    
    print('Зкончил merge')
        
    res['session_diff_time'] = (res['timestamp_last'] - res['timestamp_first'])/np.timedelta64(1, 's')
    
    print(res.shape)
    
    def f(df):
        keys, values = df.sort_values('session_id').values.T
        ukeys, index = np.unique(keys, True)
        arrays = np.split(values, index[1:])
        df2 = pd.DataFrame({'session_id':ukeys, 'timezone':[list(a) for a in arrays]})
        
        return df2
        
    buf = f(df[['session_id', 'timezone']].drop_duplicates())
    
    buf['timezone_num'] = buf['timezone'].apply(len)
    buf['timezone'] = buf['timezone'].apply(lambda x: ", ".join(set([el.strip() for el in x])))
    
    res = res.join(buf.set_index('session_id'))
    
    res.reset_index(inplace=True)
    res.rename(columns={'client':'client_pin'}, inplace=True)
            
    return res

In [78]:
%%time

parquet_ftrs = gen_features(parquet, dict_cols_categ)

Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(1004032, 235)
CPU times: user 3min 26s, sys: 41.2 s, total: 4min 8s
Wall time: 4min 8s


In [79]:
parquet['timestamp'] = pd.to_datetime(parquet['timestamp'] )

In [80]:
parquet.sort_values(by='timestamp', inplace=True)

In [81]:
parquet[parquet['session_id'] == '000011b33bb3d581fbbbcdadccb1d18e']

,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
8981804,2020-06-07 14:03:32.443,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,sv,None,None,None,SignInActivity,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
1178728,2020-06-07 14:03:32.444,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,Application Lifecycle,Foreground,None,None,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
8240620,2020-06-07 14:03:36.397,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,sv,None,None,None,MainListOnWidgetsFragment,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
3708631,2020-06-07 14:03:36.852,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,Widget Dashboard,Error,4930579a044a6143726f5a2dfca83203,Widget Dashboard,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
2694142,2020-06-07 14:03:38.450,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,Bottom Navigation,Click > Statements screen Tab,None,Main list,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
9591926,2020-06-07 14:03:38.528,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,sv,None,None,None,StatementFragment,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
3315115,2020-06-07 14:03:39.781,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,SingleStatement,load statement bonus,7a4e59a5d9e3bf6484ec3ca5e9f7e277,Single Statement,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
3315117,2020-06-07 14:03:40.066,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,SingleStatement,load statement bonus,7a4e59a5d9e3bf6484ec3ca5e9f7e277,Single Statement,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE
720604,2020-06-07 14:03:46.822,mobile,0b1b1c9ee76ef3b90c8f7f7f7877f85e,000011b33bb3d581fbbbcdadccb1d18e,se,Application Lifecycle,Background,None,None,Asia/Ho_Chi_Minh,True,None,None,mobile,LTE


In [82]:
pd.options.display.max_columns = 300

In [83]:
parquet_ftrs.head(n=1)

,session_id,client_pin,timestamp_first,timestamp_last,application_id_first,application_id_last,event_type_first,event_type_last,event_category_first,event_category_last,event_name_first,event_name_last,event_label_first,event_label_last,device_screen_name_first,device_screen_name_last,timezone_first,timezone_last,device_is_webview_first,device_is_webview_last,page_urlhost_first,page_urlhost_last,page_urlpath_full_first,page_urlpath_full_last,net_connection_type_first,net_connection_type_last,net_connection_tech_first,net_connection_tech_last,session_id_size,device_is_webview_mean,application_id_anketa_mean,application_id_mobile_mean,application_id_other_mean,application_id_retail-marketplace_mean,application_id_retail-taxes-ui_mean,application_id_retail_cards_activation_mean,application_id_retail_cards_insurance_antifraud_sale_mean,application_id_retail_cards_pin_change_mean,application_id_retail_debit_card_order_mean,application_id_retail_realty_insurance_mean,application_id_retail_upsale_cards_mean,application_id_retail_upsale_ui_mean,application_id_retail_upsale_verification_mean,application_id_site_mean,event_type_other_mean,event_type_pv_mean,event_type_se_mean,event_type_sv_mean,event_type_ue_mean,event_category_Accounts_Cards_mean,event_category_Advice_mean,event_category_All_Payments_mean,event_category_AppStoreReview_mean,event_category_Application_Lifecycle_mean,event_category_Bank_Offer_mean,event_category_Bonuses_mean,event_category_Bottom_Navigation_mean,event_category_Card_To_Card_Transfer_mean,event_category_Chat_mean,event_category_ClientPhoneTransfer_mean,event_category_Confirmation_mean,event_category_Credit_Info_mean,event_category_DeepLink_mean,event_category_ExternalPhoneTransfer_mean,event_category_Investments_mean,event_category_Investments_Bonds_mean,event_category_Main_Screen_mean,event_category_Notification_Center_push_mean,event_category_Operations_mean,event_category_Payment_Hub_mean,event_category_Phone_Bank_Picker_mean,event_category_Phone_Bank_Transfer_Confirmation_mean,event_category_Push_mean,event_category_Recharge_mean,event_category_ResultScreen_mean,event_category_ReverseCashBack_mean,event_category_SingleStatement_mean,event_category_Widget_Dashboard_mean,event_category_nan_mean,event_category_other_mean,event_name_3ds_confirm_mean,event_name_Account_info_click_mean,event_name_Account_item_click_mean,event_name_Background_mean,event_name_Choose_card_mean,event_name_Click_>_Account_Item_Info_mean,event_name_Click_>_Add_New_mean,event_name_Click_>_All_payments_screen_Tab_mean,event_name_Click_>_Chat_screen_Tab_mean,event_name_Click_>_Main_list_Tab_mean,event_name_Click_>_Save_mean,event_name_Click_>_Select_mean,event_name_Click_>_Statements_screen_Tab_mean,event_name_Click_transfer_button_mean,event_name_Confirm_code_inputted_correctly_mean,event_name_Confirm_screen_opened_mean,event_name_Expand_Accounts_mean,event_name_Force_refresh_mean,event_name_Foreground_mean,event_name_Load_individual_accounts_catalogue_mean,event_name_Load_main_PIFs_mean,event_name_Mobile_Recharge_mean,event_name_Mobile_Recharge_click_mean,event_name_Open_mean,event_name_Open_detail_mean,event_name_P2P_click_mean,event_name_Payment_finish_mean,event_name_Phone_transfer_screen_opened_mean,event_name_Preselect_Accounts_mean,event_name_Recharge_click_mean,event_name_Request_History_mean,event_name_Request_History_Result_>_Success_mean,event_name_Select_mean,event_name_Select_one_of_last_transfer_mean,event_name_Send_click_mean,event_name_Simple_screen_auto_payment_show_mean,event_name_Simple_screen_close_click_mean,event_name_Simple_screen_show_template_mean,event_name_Simple_screen_show_todo_mean,event_name_Start_drag_and_drop_mean,event_name_Submit_>_Transfer_mean,event_name_Success_>_Payment_mean,event_name_Success_>_Sms_Confirm_mean,event_name_Transfer_between_account_click_mean,event_name_get_3ds_result_mean,event_name_load_statement_bonus_mean,event_name_nan_mean,event_name_other_mean,event_name_receive_mean,event

In [11]:
train_test_df = pd.read_csv('alfabattle2_abattle_train_target.csv')
# parquet_ftrs = pd.read_pickle('/data/edmitrie/alfa_boosters/parquets_ready/part-00000.pickle')

In [12]:
# train_test_df.reset_index(inplace=True)
# parquet_ftrs.reset_index(inplace=True)
train_test_df.set_index(['client_pin', 'session_id'], inplace=True)
parquet_ftrs.set_index(['client_pin', 'session_id'], inplace=True)

In [13]:
%%time
df = parquet_ftrs.join(train_test_df)
# df = df[df.timestamp.notnull()]

CPU times: user 3.22 s, sys: 966 ms, total: 4.18 s
Wall time: 4.19 s


In [14]:
df.shape

(1004032, 239)

In [15]:
# df.drop(columns='index', inplace=True)

In [16]:
df.loc[df['multi_class_target'].isnull(), 'multi_class_target'] = 'other'

In [17]:
SEED = 1

In [19]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED)

In [20]:
atrs_to_drop = ['session_id', 'client_pin', 'multi_class_target', '_merge', 'month'] + [col for col in train_df.columns if 'timestamp' in col]

In [21]:
atrs_to_drop

['session_id',
 'client_pin',
 'multi_class_target',
 '_merge',
 'month',
 'timestamp_first',
 'timestamp_last',
 'timestamp']

In [22]:
cat_atrs = [col for col in train_df.columns if 'target' in col and 'multi' not in col and 'seq' not in col] + ['timezone', 'timezone_first', 'timezone_last'] + [col for col in train_df.columns if ('event' in col) or ('page' in col) or ('device' in col) or ('net_connection' in col) or ('application') in col]

In [23]:
cat_atrs = [col for col in cat_atrs if 'mean' not in col and 'sum' not in col]

In [24]:
atrs = [atr for atr in train_df.columns if atr not in atrs_to_drop]

In [25]:
atrs

['application_id_first',
 'application_id_last',
 'event_type_first',
 'event_type_last',
 'event_category_first',
 'event_category_last',
 'event_name_first',
 'event_name_last',
 'event_label_first',
 'event_label_last',
 'device_screen_name_first',
 'device_screen_name_last',
 'timezone_first',
 'timezone_last',
 'device_is_webview_first',
 'device_is_webview_last',
 'page_urlhost_first',
 'page_urlhost_last',
 'page_urlpath_full_first',
 'page_urlpath_full_last',
 'net_connection_type_first',
 'net_connection_type_last',
 'net_connection_tech_first',
 'net_connection_tech_last',
 'session_id_size',
 'device_is_webview_mean',
 'application_id_anketa_mean',
 'application_id_mobile_mean',
 'application_id_other_mean',
 'application_id_retail-marketplace_mean',
 'application_id_retail-taxes-ui_mean',
 'application_id_retail_cards_activation_mean',
 'application_id_retail_cards_insurance_antifraud_sale_mean',
 'application_id_retail_cards_pin_change_mean',
 'application_id_retail_debit_

In [96]:
for cat in cat_atrs:
    train_df.loc[train_df[cat].isnull(), cat] = 'nan'
    test_df.loc[test_df[cat].isnull(), cat] = 'nan'

/home/edmitrie/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


### LightGBM

In [97]:
def evalerror(y_true, y_pred):
    preds = np.reshape(y_pred, (y_true.shape[0], 11), order='F')
    preds = preds.argmax(axis = 1)
    f_score = f1_score(y_true, preds, average = 'macro')
    
    return 'f1_score', f_score, True

In [98]:
for cat_atr in cat_atrs:
    train_df[cat_atr] = train_df[cat_atr].astype('category')
    test_df[cat_atr] = test_df[cat_atr].astype('category')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
# new_atrs = [atr for atr in atrs if 'sum' not in atr and 'application_id' not in atr and 'page_urlpath' not in atr and 'page_urlhost' not in atr]

In [100]:
SEED = 1

In [101]:
lgbm = LGBMClassifier(objective='multiclass', reg_alpha=0.9, max_depth=5, min_split_gain = 0.7, subsample=0.7, n_estimators=500, n_jobs=60, random_state=SEED)

In [102]:
lgbm.fit(train_df[atrs], train_df['multi_class_target'], eval_set=(test_df[atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=50)

/home/edmitrie/.local/lib/python3.6/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/edmitrie/.local/lib/python3.6/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
/home/edmitrie/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training until validation scores don't improve for 50 rounds
[5]	valid_0's multi_logloss: 0.413744	valid_0's f1_score: 0.951471
[10]	valid_0's multi_logloss: 0.197185	valid_0's f1_score: 0.958385
[15]	valid_0's multi_logloss: 0.108032	valid_0's f1_score: 0.961127
[20]	valid_0's multi_logloss: 0.0676793	valid_0's f1_score: 0.963327
[25]	valid_0's multi_logloss: 0.0478134	valid_0's f1_score: 0.964958
[30]	valid_0's multi_logloss: 0.0372	valid_0's f1_score: 0.965986
[35]	valid_0's multi_logloss: 0.0309631	valid_0's f1_score: 0.966735
[40]	valid_0's multi_logloss: 0.0271111	valid_0's f1_score: 0.967802
[45]	valid_0's multi_logloss: 0.0246936	valid_0's f1_score: 0.968045
[50]	valid_0's multi_logloss: 0.0229652	valid_0's f1_score: 0.968653
[55]	valid_0's multi_logloss: 0.0218027	valid_0's f1_score: 0.969203
[60]	valid_0's multi_logloss: 0.0208806	valid_0's f1_score: 0.96955
[65]	valid_0's multi_logloss: 0.020235	valid_0's f1_score: 0.970073
[70]	valid_0's multi_logloss: 0.0197459	valid_0's f

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=500, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=0.9,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [103]:
pickle.dump(lgbm, open('session_parq_classifier.pickle','wb'))

In [104]:
pickle.dump(cat_atrs, open('cat_atr_parq_clas.pickle','wb'))

In [105]:
pickle.dump(atrs, open('atr_parq_clas.pickle','wb'))

## Get atrs for parquets

In [106]:
%%time

import os

P_DIR = '/data/edmitrie/alfa_boosters/parquets/'
P_DIR_ATR = '/data/edmitrie/alfa_boosters/parquets_ready/'

for p in os.listdir(P_DIR):
    print(p)
    
    parquet = pd.read_parquet(P_DIR + p)
    
    parquet.loc[parquet['device_is_webview'].isnull(), 'device_is_webview'] = False
    parquet_atrs = gen_features(parquet, dict_cols_categ)
    
    print(np.sum(parquet_atrs['timestamp_first'] > parquet_atrs['timestamp_last']))
    
    parquet_atrs.to_pickle(P_DIR_ATR + p.split('.')[0] + '.pickle')

part-00009.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(959355, 235)
0
part-00000.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(1004032, 235)
0
part-00001.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(976337, 235)
0
part-00003.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(977776, 235)
0
part-00002.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(942882, 235)
0
part-00004.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(978697, 235)
0
part-00005.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(950324, 235)
0
part-00006.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(951550, 235)
0
part-00007.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(988388, 235)
0
part-00008.parquet
Зкончил 1 агрегирование
Зкончил 2 агрегирование
Зкончил merge
(947159, 235)
0


In [3]:
import pickle

cat_atrs = pickle.load(open('cat_atr_parq_clas.pickle','rb'))
lgbm = pickle.load(open('session_parq_classifier.pickle','rb'))
atrs = pickle.load(open('atr_parq_clas.pickle','rb'))

In [108]:
atrs

['application_id_first',
 'application_id_last',
 'event_type_first',
 'event_type_last',
 'event_category_first',
 'event_category_last',
 'event_name_first',
 'event_name_last',
 'event_label_first',
 'event_label_last',
 'device_screen_name_first',
 'device_screen_name_last',
 'timezone_first',
 'timezone_last',
 'device_is_webview_first',
 'device_is_webview_last',
 'page_urlhost_first',
 'page_urlhost_last',
 'page_urlpath_full_first',
 'page_urlpath_full_last',
 'net_connection_type_first',
 'net_connection_type_last',
 'net_connection_tech_first',
 'net_connection_tech_last',
 'session_id_size',
 'device_is_webview_mean',
 'application_id_anketa_mean',
 'application_id_mobile_mean',
 'application_id_other_mean',
 'application_id_retail-marketplace_mean',
 'application_id_retail-taxes-ui_mean',
 'application_id_retail_cards_activation_mean',
 'application_id_retail_cards_insurance_antifraud_sale_mean',
 'application_id_retail_cards_pin_change_mean',
 'application_id_retail_debit_

In [114]:
%%time
res_pred_taget_df = pd.DataFrame()

import os

P_DIR_ATR = '/data/edmitrie/alfa_boosters/parquets_ready/'

cols_to_take = list(set(['client_pin', 'session_id', 'predicted_target','timestamp_first', 'timestamp_last', 'parquet_name', 'session_id_size', 'session_diff_time'] + atrs))

for p in os.listdir(P_DIR_ATR):
    print(p)
    
    parquet_atrs = pd.read_pickle(P_DIR_ATR + p)
    
    print(parquet_atrs['timestamp_first'].dtype)
    
    for cat_atr in cat_atrs:
        parquet_atrs[cat_atr] = parquet_atrs[cat_atr].astype('category')
    
    parquet_atrs['predicted_target'] = lgbm.predict(parquet_atrs[atrs])
    parquet_atrs['parquet_name'] = p
    
    buf = pd.DataFrame(lgbm.predict_proba(parquet_atrs[atrs]), columns=['proba_' + df_all for df_all in lgbm.classes_])
    res = pd.concat([parquet_atrs[cols_to_take], buf], axis=1)
    
    res_pred_taget_df = pd.concat([res_pred_taget_df, res])

part-00009.pickle
datetime64[ns]
part-00000.pickle
datetime64[ns]
part-00001.pickle
datetime64[ns]
part-00003.pickle
datetime64[ns]
part-00002.pickle
datetime64[ns]
part-00004.pickle
datetime64[ns]
part-00005.pickle
datetime64[ns]
part-00006.pickle
datetime64[ns]
part-00007.pickle
datetime64[ns]
part-00008.pickle
datetime64[ns]
CPU times: user 1h 46s, sys: 6min 58s, total: 1h 7min 44s
Wall time: 11min 32s


In [63]:
parquet_atrs = pd.read_pickle( '/data/edmitrie/alfa_boosters/parquets_ready/part-00001.pickle')

In [66]:
for cat_atr in cat_atrs:
        parquet_atrs[cat_atr] = parquet_atrs[cat_atr].astype('category')

In [67]:
parquet_atrs['predicted_target'] = lgbm.predict(parquet_atrs[atrs])
# parquet_atrs['parquet_name'] = p

NameError: name 'p' is not defined

In [68]:
parquet_atrs['parquet_name'] = 'part-00001.pickle'

In [77]:
cols_to_take = list(set(['client_pin', 'session_id', 'predicted_target','timestamp_first', 'timestamp_last', 'parquet_name', 'session_id_size', 'session_diff_time'] + atrs))

In [78]:
res = pd.concat([parquet_atrs[cols_to_take], buf], axis=1)

In [79]:
res_pred_taget_df = pd.concat([res_pred_taget_df, res])

KeyboardInterrupt: 

In [115]:
res_pred_taget_df.to_pickle('/data/edmitrie/alfa_boosters/parquet_with_target_ready_all_atrs.pickle')

## Генерация фичей для освного классификатора

In [35]:
res_pred_taget_df = pd.read_pickle('/data/edmitrie/alfa_boosters/parquet_with_target_ready_all_atrs.pickle')

In [36]:
train_test_df = pd.read_csv('alfabattle2_abattle_train_target.csv')
submit_df = pd.read_csv('alfabattle2_prediction_session_timestamp.csv')

In [37]:
df_train = train_test_df.drop(columns='timestamp')

In [38]:
%%time
subm_parq_df = pd.concat([submit_df, res_pred_taget_df])

CPU times: user 23.8 s, sys: 40.3 s, total: 1min 4s
Wall time: 1min 4s


In [39]:
%%time
df_all = pd.merge(subm_parq_df, df_train, on=['client_pin','session_id'], how='left')

CPU times: user 26.2 s, sys: 18.6 s, total: 44.8 s
Wall time: 44.9 s


In [40]:
df_all['timestamp'] = pd.to_datetime(df_all['timestamp'])

In [41]:
df_all.loc[df_all.session_id.isnull(), 'timestamp_first'] = df_all.loc[df_all.session_id.isnull(), 'timestamp']

In [42]:
df_all.sort_values(by='timestamp_first', inplace=True)

### Тест что в ремя в сабмите поздее паркетов, тут все ок, можно пропускать блок

In [34]:
testing = pd.merge(submit_df, res_pred_taget_df, on='client_pin', how='left')

In [35]:
testing = testing[testing['timestamp'].notnull()]

In [47]:
testing[testing['client_pin'] == 'f0c674b2bb4dc64be607029271d706ec']

,client_pin,timestamp,device_screen_name_PaymentHubActivity_mean,device_screen_name_ChatNewFragment_mean,event_name_Select_one_of_last_transfer_mean,device_screen_name_Statements_screen_mean,device_screen_name_Add_New_Card_mean,event_label_03e1bc37c7331a5995071f103977aaeb_mean,event_category_Notification_Center_push_mean,event_name_Click_>_Account_Item_Info_mean,...,proba_card_recharge,proba_chat,proba_credit_info,proba_invest,proba_main_screen,proba_mobile_recharge,proba_other,proba_own_transfer,proba_phone_money_transfer,proba_statement
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.000001,0.000003,9.999410e-01,0.000002,6.389368e-07,3.651443e-07,0.000046,7.421889e-07,0.000004,2.156997e-07
1,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.999527,0.000231,1.904146e-07,0.000003,6.544126e-10,1.870176e-07,0.000230,3.191099e-07,0.000007,4.040358e-07
2,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.999605,0.000001,9.513196e-08,0.000001,1.314681e-09,8.267707e-08,0.000387,1.239667e-07,0.000005,1.050767e-07
3,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.000001,0.000003,9.999413e-01,0.000001,6.389370e-07,4.112395e-07,0.000046,6.494864e-07,0.000004,2.115836e-07
4,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000007,0.000010,2.640741e-06,0.000024,1.064012e-05,1.940812e-06,0.999910,4.574654e-06,0.000012,1.193612e-05
5,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000007,0.000010,2.640741e-06,0.000024,1.064012e-05,1.940812e-06,0.999910,4.574654e-06,0.000012,1.193612e-05
6,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000015,0.000034,4.192574e-06,0.000051,2.846364e-07,3.780640e-06,0.999811,7.411365e-06,0.000044,2.025264e-05


In [53]:
testing['timestamp_first'].isnull().sum()

0

In [54]:
testing['timestamp'].isnull().sum()

0

In [56]:
np.sum(testing['timestamp_first'] > pd.to_datetime(testing['timestamp']))

0

### Продолжение генерации фичей

In [44]:
top_targets = 5

In [45]:
for i in range(1, top_targets+1):
    df_all[f'last_target_parq_{i}'] = df_all.groupby('client_pin')['predicted_target'].shift(i)

In [46]:
buf = pd.get_dummies(df_all['predicted_target'], prefix='prev_sum_parq')

In [47]:
cum_cols = buf.columns

In [48]:
df_all = pd.concat([df_all, buf], axis=1)

In [49]:
df_all[cum_cols] = df_all.groupby('client_pin')[cum_cols].cumsum()
df_all[cum_cols] = df_all.groupby('client_pin')[cum_cols].shift(1)

In [50]:
for i in range(1, top_targets+1):
    df_all[f'top_target_parq_{i}'] = cum_cols[df_all[cum_cols].values.argsort(1)[:, -i]]
    df_all[f'top_target_parq_{i}'] = df_all[f'top_target_parq_{i}'].apply(lambda x: x.split('prev_sum_')[1])

In [53]:
# last_cols = [col for col in df_all.columns if 'proba' in col or 'first' in col or 'last' in col] + ['session_diff_time','session_id_size']
last_cols = atrs

In [54]:
num_last_cols = 2

In [55]:
%%time

for i in range(1, num_last_cols +1):
    df_all[[col + f'_prev_{i}' for col in last_cols]] = df_all.groupby('client_pin')[last_cols].shift(i)

KeyboardInterrupt: 

In [138]:
len(atrs)

235

In [1650]:
# df_all[last_cols] = df_all.groupby('client_pin')[last_cols].shift(1)

In [66]:
# cols_to_take = [col for col in df_all.columns if 'top' in col or 'last' in col and 'timestamp' not in col  or 'prev' in col] + last_cols + ['client_pin', 'session_id']
# cols_to_take = [col for col in df_all.columns if 'top' in col or 'last' in col and '  or 'prev' in col] + ['client_pin', 'session_id']

In [142]:
def get_mean_col(df, col):
    df[f'mean_{col}_parq'] = df.groupby('client_pin')[col].cumsum()
    df[f'mean_{col}_parq'] = df.groupby('client_pin')[col].shift(1)
    df[f'mean_{col}_parq'] /= df.groupby('client_pin')[col].cumcount()
    
    return df

In [144]:
df_all = get_mean_col(df_all, 'session_id_size')
df_all = get_mean_col(df_all, 'session_diff_time')

In [152]:
df_all['time_diff_1_0_parq'] = df_all['session_diff_time_prev_1'] - df_all['session_diff_time_prev_2']
df_all['session_id_size_1_0_parq'] = df_all['session_id_size_prev_1'] - df_all['session_id_size_prev_2']

In [52]:
cols_to_take = list(set([col for col in df_all.columns if col not in last_cols + ['timestamp']]))

In [53]:
len(cols_to_take)

513

In [56]:
df_all[cols_to_take].to_pickle('/data/edmitrie/alfa_boosters/parquet_full_ready_all_cols.pickle')

In [160]:
df_all.to_pickle('/data/edmitrie/alfa_boosters/parquet_super_full_ready_all_cols.pickle')

In [7]:
import pandas as pd

In [8]:
df_all = pd.read_pickle('/data/edmitrie/alfa_boosters/parquet_super_full_ready_all_cols.pickle')

In [10]:
cols_to_agg = [col for col in atrs if 'mean' in col]

In [11]:
new_cols_mean = ['mean_parq_all_sessions' + el for el in cols_to_agg]

In [12]:
df_all[new_cols_mean] = df_all[cols_to_agg] > 0

In [13]:
df_all.head(n=1)

,client_pin,timestamp,device_screen_name_PaymentHubActivity_mean,device_screen_name_ChatNewFragment_mean,event_name_Select_one_of_last_transfer_mean,device_screen_name_Statements_screen_mean,device_screen_name_Add_New_Card_mean,event_label_03e1bc37c7331a5995071f103977aaeb_mean,event_category_Notification_Center_push_mean,event_name_Click_>_Account_Item_Info_mean,...,mean_parq_all_sessionsnet_connection_tech_HSDPA_mean,mean_parq_all_sessionsnet_connection_tech_HSPA_mean,mean_parq_all_sessionsnet_connection_tech_HSPA+_mean,mean_parq_all_sessionsnet_connection_tech_HSUPA_mean,mean_parq_all_sessionsnet_connection_tech_LTE_mean,mean_parq_all_sessionsnet_connection_tech_LTE_CA_mean,mean_parq_all_sessionsnet_connection_tech_UMTS_mean,mean_parq_all_sessionsnet_connection_tech_UNKNOWN_mean,mean_parq_all_sessionsnet_connection_tech_nan_mean,mean_parq_all_sessionsnet_connection_tech_other_mean
7796369,989f645a00e3e8a179036cf5fd5be29d,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False


In [15]:
%%time

for col in new_cols_mean:
    print(col)
    df_all[col] = df_all.groupby('client_pin')[col].cumsum()

mean_parq_all_sessionsdevice_is_webview_mean
mean_parq_all_sessionsapplication_id_anketa_mean
mean_parq_all_sessionsapplication_id_mobile_mean
mean_parq_all_sessionsapplication_id_other_mean
mean_parq_all_sessionsapplication_id_retail-marketplace_mean
mean_parq_all_sessionsapplication_id_retail-taxes-ui_mean
mean_parq_all_sessionsapplication_id_retail_cards_activation_mean
mean_parq_all_sessionsapplication_id_retail_cards_insurance_antifraud_sale_mean
mean_parq_all_sessionsapplication_id_retail_cards_pin_change_mean
mean_parq_all_sessionsapplication_id_retail_debit_card_order_mean
mean_parq_all_sessionsapplication_id_retail_realty_insurance_mean
mean_parq_all_sessionsapplication_id_retail_upsale_cards_mean
mean_parq_all_sessionsapplication_id_retail_upsale_ui_mean
mean_parq_all_sessionsapplication_id_retail_upsale_verification_mean
mean_parq_all_sessionsapplication_id_site_mean
mean_parq_all_sessionsevent_type_other_mean
mean_parq_all_sessionsevent_type_pv_mean
mean_parq_all_sessionsev

In [16]:
%%time

for col in new_cols_mean:
    print(col)
    df_all[col] = df_all.groupby('client_pin')[col].shift(1)/df_all.groupby('client_pin')[col].cumcount()

mean_parq_all_sessionsdevice_is_webview_mean
mean_parq_all_sessionsapplication_id_anketa_mean
mean_parq_all_sessionsapplication_id_mobile_mean
mean_parq_all_sessionsapplication_id_other_mean
mean_parq_all_sessionsapplication_id_retail-marketplace_mean
mean_parq_all_sessionsapplication_id_retail-taxes-ui_mean
mean_parq_all_sessionsapplication_id_retail_cards_activation_mean
mean_parq_all_sessionsapplication_id_retail_cards_insurance_antifraud_sale_mean
mean_parq_all_sessionsapplication_id_retail_cards_pin_change_mean
mean_parq_all_sessionsapplication_id_retail_debit_card_order_mean
mean_parq_all_sessionsapplication_id_retail_realty_insurance_mean
mean_parq_all_sessionsapplication_id_retail_upsale_cards_mean
mean_parq_all_sessionsapplication_id_retail_upsale_ui_mean
mean_parq_all_sessionsapplication_id_retail_upsale_verification_mean
mean_parq_all_sessionsapplication_id_site_mean
mean_parq_all_sessionsevent_type_other_mean
mean_parq_all_sessionsevent_type_pv_mean
mean_parq_all_sessionsev

In [18]:
cols_to_take = list(set([col for col in df_all.columns if col not in atrs + ['timestamp']]))

In [20]:
len(cols_to_take)

720

In [21]:
df_all[cols_to_take].to_pickle('/data/edmitrie/alfa_boosters/parquet_full_ready_all_cols.pickle')